In [ ]:
# default_exp data.token_classification

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# data.token_classification

> This module contains the bits required to use the fastai DataBlock API and/or mid-level data processing pipelines to organize your data for token classification tasks (e.g., NER or named entity recognition, etc...).

In [ ]:
#export
import ast
from functools import reduce

import torch
from transformers import *
from fastai.text.all import *

from blurr.utils import *
from blurr.data.core import *

In [ ]:
#hide
import pdb

from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

Using GPU #1: GeForce GTX 1080 Ti


## Token classification tokenization, batch transform, and DataBlock methods

Token classification tasks attempt to predict a class for each token.  The idea is similar to that in image segmentation models where the objective is to predict a class for each pixel.  Such models are common in building named entity recognition (NER) systems.

In [ ]:
# ensures these cols are represented as lists (rather than string)
df_converters = {'tokens': ast.literal_eval, 'labels': ast.literal_eval, 'nested-labels': ast.literal_eval}

path = Path('./')
germ_eval_df = pd.read_csv(path/'germeval2014_sample.csv', converters=df_converters); len(germ_eval_df)

1000

In [ ]:
#hide
# for idx, el in germ_eval_df.iterrows():
#     print (el['tokens'])
#     print (el['labels'])
#     print('---------------')

In [ ]:
labels = sorted(list(set([lbls for sublist in germ_eval_df.labels.tolist() for lbls in sublist])))
print(labels)

['B-LOC', 'B-LOCderiv', 'B-LOCpart', 'B-ORG', 'B-ORGpart', 'B-OTH', 'B-OTHderiv', 'B-OTHpart', 'B-PER', 'B-PERderiv', 'B-PERpart', 'I-LOC', 'I-LOCderiv', 'I-ORG', 'I-ORGpart', 'I-OTH', 'I-PER', 'O']


In [ ]:
task = HF_TASKS_AUTO.TokenClassification

pretrained_model_name = "bert-base-multilingual-cased"
n_labels = len(labels)

hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(pretrained_model_name, 
                                                                               task=task,
                                                                               config_kwargs={'num_labels': n_labels})
hf_arch, type(hf_config), type(hf_tokenizer), type(hf_model)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

('bert',
 transformers.configuration_bert.BertConfig,
 transformers.tokenization_bert.BertTokenizer,
 transformers.modeling_bert.BertForTokenClassification)

Below, we define a new class and transform for token classification targets/predictions.

In [ ]:
#export
class HF_TokenTensorCategory(TensorBase): pass

In [ ]:
#export
class HF_TokenCategorize(Transform):
    "Reversible transform of a list of category string to `vocab` id"
    
    def __init__(self, vocab=None, ignore_token=None, ignore_token_id=None):  
        self.vocab = None if vocab is None else CategoryMap(vocab)
        self.ignore_token = '[xIGNx]' if ignore_token is None else ignore_token
        self.ignore_token_id = CrossEntropyLossFlat().ignore_index if ignore_token_id is None else ignore_token_id
        
        self.loss_func, self.order = CrossEntropyLossFlat(ignore_index=self.ignore_token_id), 1

    def setups(self, dsets):
        if self.vocab is None and dsets is not None: self.vocab = CategoryMap(dsets)
        self.c = len(self.vocab)

    def encodes(self, labels):
        ids = [[self.vocab.o2i[lbl]] + [self.ignore_token_id]*(n_subtoks-1) for lbl, n_subtoks in labels] 
        return HF_TokenTensorCategory(reduce(operator.concat, ids))
    
    def decodes(self, encoded_labels): 
        return Category([(self.vocab[lbl_id]) for lbl_id in encoded_labels if lbl_id != self.ignore_token_id ])

`HF_TokenCategorize` modifies the fastai `Categorize` transform in a couple of ways.  First, it allows your targets to consist of a `Category` ***per*** token, and second, it uses the idea of an `ignore_token` to mask subtokens that don't need a prediction.  For example, the target of special tokens (e.g., pad, cls, sep) are set to `ignore_token` as are subsequent sub-tokens of a given token should more than 1 sub-token make it up.

In [ ]:
#export
def HF_TokenCategoryBlock(vocab=None, ignore_token=None, ignore_token_id=None):
    "`TransformBlock` for single-label categorical targets"
    return TransformBlock(type_tfms=HF_TokenCategorize(vocab=vocab, 
                                                       ignore_token=ignore_token,
                                                       ignore_token_id=ignore_token_id))

In [ ]:
show_doc(HF_TokenCategoryBlock)

<h4 id="HF_TokenCategoryBlock" class="doc_header"><code>HF_TokenCategoryBlock</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>HF_TokenCategoryBlock</code>(**`vocab`**=*`None`*, **`ignore_token`**=*`None`*, **`ignore_token_id`**=*`None`*)

`TransformBlock` for single-label categorical targets

Again, we define a custom class, `HF_TokenClassInput`, for the @typedispatched methods to use so that we can override how token classification inputs/targets are assembled, as well as, how the data is shown via methods like `show_batch` and `show_results`.

In [ ]:
#export
class HF_TokenClassInput(HF_BaseInput): pass

In [ ]:
#export
class HF_TokenClassBatchTransform(HF_BatchTransform):
    def __init__(self, hf_arch, hf_tokenizer, 
                 ignore_token_id=CrossEntropyLossFlat().ignore_index,
                 max_length=None, padding=True, truncation=True, is_pretokenized=True,
                 n_tok_inps=1, hf_input_return_type=HF_TokenClassInput, tok_kwargs={}, **kwargs):
                 
        super().__init__(hf_arch, hf_tokenizer,
                         max_length=max_length, padding=padding, truncation=truncation, is_pretokenized=is_pretokenized,
                         n_tok_inps=n_tok_inps, hf_input_return_type=hf_input_return_type, tok_kwargs=tok_kwargs, **kwargs)

        self.ignore_token_id = ignore_token_id
        
    def encodes(self, samples):  
        samples = super().encodes(samples)
        if (len(samples[0]) == 1): return samples
        
        target_cls = type(samples[0][1])
        updated_samples = []
        
        # we assume that first target = the categories we want to predict for each token
        for s in samples:
            targ_len = len(s[1])
            idx_first_input_id = s[0]['special_tokens_mask'].tolist().index(0)
            targ_ids = target_cls([ self.ignore_token_id if (el == 1 or idx > targ_len) 
                                   else s[1][idx-idx_first_input_id].item() 
                                   for idx, el in enumerate(s[0]['special_tokens_mask']) ])

            updated_samples.append((s[0], targ_ids))
        
        return updated_samples

`HF_TokenClassBatchTransform` is used to turn any targets we don't want to include in the loss calcuation (e.g. padding, cls, sep, etc...).

In [ ]:
hf_batch_tfm = HF_TokenClassBatchTransform(hf_arch, hf_tokenizer, is_pretokenized=True, tok_kwargs={ 'return_special_tokens_mask': True })

blocks = (
    HF_TextBlock(hf_batch_tfm=hf_batch_tfm), 
    HF_TokenCategoryBlock(vocab=labels)
)

def get_y(inp):
    return [ (label, len(hf_tokenizer.tokenize(str(entity)))) for entity, label in zip(inp.tokens, inp.labels) ]

dblock = DataBlock(blocks=blocks, 
                   get_x=ColReader('tokens'),
                   get_y=get_y,
                   splitter=RandomSplitter())

Note in the example above we had to define a `get_y` in order to return both the entity we want to predict a category for, as well as, how many subtokens are used by the `hf_tokenizer` to represent it.  This is necessary for the input/target alignment discussed above.

In [ ]:
# dblock.summary(test_df)

In [ ]:
dls = dblock.dataloaders(germ_eval_df, bs=4)

In [ ]:
b = dls.one_batch()

In [ ]:
len(b), b[0]['input_ids'].shape, b[1].shape

(2, torch.Size([4, 76]), torch.Size([4, 76]))

In [ ]:
#export
@typedispatch
def show_batch(x:HF_TokenClassInput, y, samples, dataloaders, ctxs=None, max_n=6, **kwargs):  
    hf_tokenizer = dataloaders.before_batch[0].hf_tokenizer
    
    res = L()
    for inp, trg, sample in zip(x, y, samples):
        # recontstruct the string and split on space to get back your pre-tokenized list of tokens
        toks = hf_tokenizer.convert_ids_to_tokens(inp, skip_special_tokens=True)
        pretokenized_toks =  hf_tokenizer.convert_tokens_to_string(toks).split()

        res.append([f'{[ (tok, lbl) for tok, lbl in zip(pretokenized_toks, ast.literal_eval(sample[1])) ]}'])
        
    display_df(pd.DataFrame(res, columns=['token / target label'])[:max_n])
    return ctxs

In [ ]:
dls.show_batch(dataloaders=dls, max_n=2)

,token / target label
0,"[('Helbig', 'B-OTH'), ('et', 'I-OTH'), ('al', 'I-OTH'), ('.', 'O'), ('(', 'O'), ('1994', 'O'), (')', 'O'), ('S', 'O'), ('.', 'O'), ('593', 'B-OTH'), ('.', 'I-OTH'), ('Wink', 'I-OTH'), ('&', 'I-OTH'), ('Seibold', 'I-OTH'), ('et', 'O'), ('al', 'O'), ('.', 'O'), ('(', 'O'), ('1998', 'O'), (')', 'O'), ('S', 'O'), ('.', 'O'), ('32', 'O'), ('Inwieweit', 'O'), ('noch', 'O'), ('andere', 'O'), ('Falken', 'O'), (',', 'B-LOCderiv'), ('wie', 'O'), ('der', 'O'), ('Afrikanische', 'O'), ('Baumfalke', 'O'), ('(', 'O'), ('Falco', 'O'), ('cuvieri', 'O'), (')', 'O'), ('oder', 'O'), ('der', 'O'), ('Malaienbaumfalke', 'O'), ('(', 'O'), ('Falco', 'O'), ('serverus', 'O'), (')', 'O'), ('dieser', 'O'), ('Gruppe', 'O'), ('zuzuzählen', 'O'), ('sind', 'O'), (',', 'O'), ('ist', 'O'), ('Gegenstand', 'O')]"
1,"[('Scenes', 'B-OTH'), ('of', 'I-OTH'), ('a', 'I-OTH'), ('Sexual', 'I-OTH'), ('Nature', 'I-OTH'), ('(', 'O'), ('GB', 'O'), ('2006', 'O'), (')', 'O'), ('-', 'O'), ('Regie', 'O'), (':', 'O'), ('Ed', 'B-PER'), ('Blum', 'I-PER'), ('Shortbus', 'B-OTH'), ('(', 'O'), ('USA', 'B-LOC'), ('2006', 'O'), (')', 'O'), ('-', 'O'), ('Regie', 'O'), (':', 'O'), ('John', 'B-PER'), ('Cameron', 'I-PER'), ('Mitchell', 'I-PER'), (':', 'O'), ('Film', 'O'), ('über', 'O'), ('den', 'O'), ('gleichnamigen', 'B-LOCderiv'), ('New', 'I-LOCderiv'), ('Yorker', 'O'), ('Club', 'O'), (',', 'O'), ('der', 'O'), ('verschiedensten', 'O'), ('Paaren', 'O'), ('eine', 'O'), ('Plattform', 'O'), ('zur', 'O'), ('Aufarbeitung', 'O'), ('ihrer', 'O'), ('Probleme', 'O'), ('bietet', 'O')]"


## Tests

The tests below to ensure the core DataBlock code above works for **all** pretrained token classification models available in huggingface.  These tests are excluded from the CI workflow because of how long they would take to run and the amount of data that would be required to download.

**Note**: Feel free to modify the code below to test whatever pretrained classification models you are working with ... and if any of your pretrained token classification models fail, please submit a github issue *(or a PR if you'd like to fix it yourself)*

In [ ]:
BLURR_MODEL_HELPER.get_models(task='TokenClassification')

[transformers.modeling_albert.AlbertForTokenClassification,
 transformers.modeling_auto.AutoModelForTokenClassification,
 transformers.modeling_bert.BertForTokenClassification,
 transformers.modeling_camembert.CamembertForTokenClassification,
 transformers.modeling_distilbert.DistilBertForTokenClassification,
 transformers.modeling_electra.ElectraForTokenClassification,
 transformers.modeling_flaubert.FlaubertForTokenClassification,
 transformers.modeling_longformer.LongformerForTokenClassification,
 transformers.modeling_mobilebert.MobileBertForTokenClassification,
 transformers.modeling_roberta.RobertaForTokenClassification,
 transformers.modeling_xlm.XLMForTokenClassification,
 transformers.modeling_xlm_roberta.XLMRobertaForTokenClassification,
 transformers.modeling_xlnet.XLNetForTokenClassification]

In [ ]:
pretrained_model_names = [
    'albert-base-v1',
    'bert-base-multilingual-cased',
    'camembert-base',
    'distilbert-base-uncased',
    'monologg/electra-small-finetuned-imdb',
    'allenai/longformer-base-4096',
    'google/mobilebert-uncased',
    'roberta-base',
    'xlm-mlm-en-2048',
    'xlm-roberta-base',
    'xlnet-base-cased'
]

In [ ]:
#slow
#hide_output
task = HF_TASKS_AUTO.TokenClassification
bsz = 2
seq_sz = 128

test_results = []
for model_name in pretrained_model_names:
    error=None
    
    print(f'=== {model_name} ===\n')
    
    hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(model_name, task=task)
    print(f'architecture:\t{hf_arch}\ntokenizer:\t{type(hf_tokenizer).__name__}\n')
    
    hf_batch_tfm = HF_TokenClassBatchTransform(hf_arch, hf_tokenizer, padding='max_length', max_length=seq_sz, 
                                               is_pretokenized=True, tok_kwargs={ 'return_special_tokens_mask': True })

    blocks = (
        HF_TextBlock(hf_arch, hf_tokenizer, hf_batch_tfm=hf_batch_tfm), 
        HF_TokenCategoryBlock(vocab=labels)
    )

    dblock = DataBlock(blocks=blocks, 
                       get_x=ColReader('tokens'),
                       get_y= lambda inp: [ (label, len(hf_tokenizer.tokenize(str(entity)))) for entity, label in zip(inp.tokens, inp.labels) ],
                       splitter=RandomSplitter())
    
    dls = dblock.dataloaders(germ_eval_df, bs=bsz)
    b = dls.one_batch()
    
    try:
        print('*** TESTING DataLoaders ***\n')
        test_eq(len(b), 2)
        test_eq(len(b[0]['input_ids']), bsz)
        test_eq(b[0]['input_ids'].shape, torch.Size([bsz, seq_sz]))
        test_eq(len(b[1]), bsz)

        if (hasattr(hf_tokenizer, 'add_prefix_space')):
            test_eq(dls.before_batch[0].tok_kwargs['add_prefix_space'], True)

        test_results.append((hf_arch, type(hf_tokenizer).__name__, model_name, 'PASSED', ''))
        dls.show_batch(dataloaders=dls, max_n=2)
        
    except Exception as err:
        test_results.append((hf_arch, type(hf_tokenizer).__name__, model_name, 'FAILED', err))

=== albert-base-v1 ===

Some weights of the model checkpoint at albert-base-v1 were not used when initializing AlbertForTokenClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at albert-base-v1 and are newly initialized: ['classifier.weight', 'classifier.bi

,token / target label
0,"[('helbig', 'B-OTH'), ('et', 'I-OTH'), ('al', 'I-OTH'), ('.', 'O'), ('(', 'O'), ('1994', 'O'), (')', 'O'), ('s.', 'O'), ('593.', 'O'), ('wink', 'B-OTH'), ('&', 'I-OTH'), ('seibold', 'I-OTH'), ('et', 'I-OTH'), ('al', 'I-OTH'), ('.', 'O'), ('(', 'O'), ('1998', 'O'), (')', 'O'), ('s.', 'O'), ('32', 'O'), ('inwieweit', 'O'), ('noch', 'O'), ('andere', 'O'), ('falken', 'O'), (',', 'O'), ('wie', 'O'), ('der', 'O'), ('afrikanische', 'B-LOCderiv'), ('baumfalke', 'O'), ('(', 'O'), ('falco', 'O'), ('cuvieri', 'O'), (')', 'O'), ('oder', 'O'), ('der', 'O'), ('malaienbaumfalke', 'O'), ('(', 'O'), ('falco', 'O'), ('serverus', 'O'), (')', 'O'), ('dieser', 'O'), ('gruppe', 'O'), ('zuzuzahlen', 'O'), ('sind', 'O'), (',', 'O'), ('ist', 'O'), ('gegenstand', 'O'), ('der', 'O'), ('forschung', 'O'), ('.', 'O')]"
1,"[('auerdem', 'O'), ('befindet', 'O'), ('sich', 'O'), ('im', 'O'), ('nordwesten', 'O'), ('der', 'O'), ('stadt', 'O'), ('(', 'O'), ('auf', 'O'), ('dem', 'O'), ('gelande', 'O'), ('des', 'O'), ('ehemaligen', 'O'), ('militarflughafens', 'O'), ('butzweilerhof', 'B-LOC'), (')', 'O'), ('das', 'O'), ('coloneum', 'B-LOC'), (',', 'O'), ('europas', 'B-LOC'), ('groter', 'O'), ('studiokomplex', 'O'), ('mit', 'O'), ('einer', 'O'), ('flache', 'O'), ('von', 'O'), ('35', 'O'), ('ha', 'O'), ('und', 'O'), ('20', 'O'), ('studios', 'O'), ('(', 'O'), ('25.000', 'O'), ('m2', 'O'), (')', 'O'), ('mit', 'O'), ('bis', 'O'), ('zu', 'O'), ('30', 'O'), ('meter', 'O'), ('deckenhohe', 'O'), ('.', 'O')]"


=== bert-base-multilingual-cased ===

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not ini

,token / target label
0,"[('Helbig', 'B-OTH'), ('et', 'I-OTH'), ('al', 'I-OTH'), ('.', 'O'), ('(', 'O'), ('1994', 'O'), (')', 'O'), ('S', 'O'), ('.', 'O'), ('593', 'B-OTH'), ('.', 'I-OTH'), ('Wink', 'I-OTH'), ('&', 'I-OTH'), ('Seibold', 'I-OTH'), ('et', 'O'), ('al', 'O'), ('.', 'O'), ('(', 'O'), ('1998', 'O'), (')', 'O'), ('S', 'O'), ('.', 'O'), ('32', 'O'), ('Inwieweit', 'O'), ('noch', 'O'), ('andere', 'O'), ('Falken', 'O'), (',', 'B-LOCderiv'), ('wie', 'O'), ('der', 'O'), ('Afrikanische', 'O'), ('Baumfalke', 'O'), ('(', 'O'), ('Falco', 'O'), ('cuvieri', 'O'), (')', 'O'), ('oder', 'O'), ('der', 'O'), ('Malaienbaumfalke', 'O'), ('(', 'O'), ('Falco', 'O'), ('serverus', 'O'), (')', 'O'), ('dieser', 'O'), ('Gruppe', 'O'), ('zuzuzählen', 'O'), ('sind', 'O'), (',', 'O'), ('ist', 'O'), ('Gegenstand', 'O')]"
1,"[('Mit', 'O'), ('der', 'O'), ('Servicefrau', 'O'), ('verband', 'O'), ('Bianca', 'B-PER'), ('offenbar', 'O'), ('eine', 'O'), ('Art', 'O'), ('Freundschaft', 'O'), ('oder', 'O'), ('wenigstens', 'O'), ('wünschte', 'O'), ('sich', 'O'), ('das', 'O'), ('Bianca', 'O'), ('B', 'B-PER'), ('.', 'I-PER'), ('«', 'O'), ('Sie', 'O'), ('gab', 'O'), ('mir', 'O'), ('immer', 'O'), ('wieder', 'O'), ('Geld', 'O'), ('oder', 'O'), ('kleine', 'O'), ('Geschenke', 'O'), ('»', 'O'), (',', 'O'), ('so', 'O'), ('L', 'B-PER'), ('.', 'O'), ('Doch', 'O'), ('gleichzeitig', 'O'), ('sei', 'B-PER'), ('Bianca', 'I-PER'), ('B', 'O'), ('.', 'O'), ('ihr', 'O'), ('gegenüber', 'O'), ('nicht', 'O'), ('ehrlich', 'O')]"


=== camembert-base ===

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this mo

...

,token / target label
0,"[('helbig', 'B-OTH'), ('et', 'I-OTH'), ('al', 'I-OTH'), ('.', 'O'), ('(', 'O'), ('1994', 'O'), (')', 'O'), ('s.', 'O'), ('593', 'O'), ('.', 'B-OTH'), ('wink', 'I-OTH'), ('&', 'I-OTH'), ('seibold', 'I-OTH'), ('et', 'I-OTH'), ('al', 'O'), ('.', 'O'), ('(', 'O'), ('1998', 'O'), (')', 'O'), ('s.', 'O'), ('32', 'O'), ('inwieweit', 'O'), ('noch', 'O'), ('andere', 'O'), ('falken', 'O'), (',', 'O'), ('wie', 'O'), ('der', 'B-LOCderiv'), ('afrikanische', 'O'), ('baumfalke', 'O'), ('(', 'O'), ('falco', 'O'), ('cuvieri', 'O'), (')', 'O'), ('oder', 'O'), ('der', 'O'), ('malaienbaumfalke', 'O'), ('(', 'O'), ('falco', 'O'), ('serverus', 'O'), (')', 'O'), ('dieser', 'O'), ('gruppe', 'O'), ('zuzuzahlen', 'O'), ('sind', 'O'), (',', 'O'), ('ist', 'O'), ('gegenstand', 'O'), ('der', 'O'), ('forschung', 'O')]"
1,"[('(', 'O'), ('standard', 'B-ORG'), ('oil', 'I-ORG'), ('of', 'I-ORG'), ('new', 'I-ORG'), ('jersey', 'I-ORG'), (')', 'O'), (',', 'O'), ('die', 'O'), ('ausgesprochen', 'O'), ('""', 'O'), ('esso', 'B-ORG'), ('""', 'O'), ('ergeben', 'O'), ('(', 'O'), ('heute', 'O'), ('exxonmobil', 'B-ORG'), (')', 'O'), ('.', 'O'), (';', 'O'), ('exxon', 'B-ORG'), (':', 'O'), ('ein', 'O'), ('name', 'O'), (',', 'O'), ('der', 'O'), ('in', 'O'), ('den', 'O'), ('fruhen', 'O'), ('1970ern', 'O'), ('von', 'O'), ('esso', 'B-ORG'), ('erfunden', 'O'), ('wurde', 'O'), (',', 'O'), ('um', 'O'), ('ein', 'O'), ('neutrales', 'O'), ('aber', 'O'), ('eindeutiges', 'O'), ('markenzeichen', 'O'), ('fur', 'O'), ('das', 'O'), ('unternehmen', 'O'), ('zu', 'O'), ('haben', 'O'), ('.', 'O')]"


=== xlm-roberta-base ===

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRA

,token / target label
0,"[('(', 'O'), ('Standard', 'B-ORG'), ('Oil', 'I-ORG'), ('of', 'I-ORG'), ('New', 'I-ORG'), ('Jersey', 'I-ORG'), (')', 'O'), (',', 'O'), ('die', 'O'), ('ausgesprochen', 'O'), ('„', 'O'), ('Esso', 'B-ORG'), ('“', 'O'), ('ergeben', 'O'), ('(', 'O'), ('heute', 'O'), ('ExxonMobil', 'B-ORG'), (')', 'O'), ('.', 'O'), (';', 'O'), ('Exxon', 'B-ORG'), (':', 'O'), ('Ein', 'O'), ('Name', 'O'), (',', 'O'), ('der', 'O'), ('in', 'O'), ('den', 'O'), ('frühen', 'O'), ('1970ern', 'O'), ('von', 'O'), ('Esso', 'B-ORG'), ('erfunden', 'O'), ('wurde', 'O'), (',', 'O'), ('um', 'O'), ('ein', 'O'), ('neutrales', 'O'), ('aber', 'O'), ('eindeutiges', 'O'), ('Markenzeichen', 'O'), ('für', 'O'), ('das', 'O'), ('Unternehmen', 'O'), ('zu', 'O'), ('haben', 'O'), ('.', 'O')]"
1,"[('Die', 'O'), ('Flügel', 'O'), ('Die', 'O'), ('geöffneten', 'O'), ('Flügel', 'O'), ('zeigen', 'O'), ('in', 'O'), ('vier', 'O'), ('Szenen', 'O'), ('Höhepunkte', 'O'), ('aus', 'O'), ('dem', 'O'), ('Leben', 'O'), ('von', 'O'), ('Maria', 'B-PER'), ('und', 'O'), ('Jesus', 'B-PER'), (',', 'O'), ('passend', 'O'), ('zu', 'O'), ('den', 'O'), ('christlichen', 'O'), ('Festen', 'O'), ('Weihnachten', 'O'), (',', 'O'), ('Ostern', 'O'), ('und', 'O'), ('Pfingsten', 'O'), (':', 'O'), ('Der', 'O'), ('Hauptschrein', 'O'), ('Im', 'O'), ('Hauptschrein', 'O'), ('ist', 'O'), ('die', 'O'), ('Krönung', 'O'), ('Mariens', 'B-PER'), ('durch', 'O'), ('Christus', 'B-PER'), ('dargestellt', 'O'), ('.', 'O')]"


=== xlnet-base-cased ===

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForTokenClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLNetForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForTokenClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
architecture:	xlnet
tokenizer:	XLNetTokeni

,token / target label
0,"[('Helbig', 'B-OTH'), ('et', 'I-OTH'), ('al', 'I-OTH'), ('.', 'O'), ('(', 'O'), ('1994', 'O'), (')', 'O'), ('S.', 'O'), ('593.', 'O'), ('Wink', 'B-OTH'), ('&', 'I-OTH'), ('Seibold', 'I-OTH'), ('et', 'I-OTH'), ('al', 'I-OTH'), ('.', 'O'), ('(', 'O'), ('1998', 'O'), (')', 'O'), ('S.', 'O'), ('32', 'O'), ('Inwieweit', 'O'), ('noch', 'O'), ('andere', 'O'), ('Falken', 'O'), (',', 'O'), ('wie', 'O'), ('der', 'O'), ('Afrikanische', 'B-LOCderiv'), ('Baumfalke', 'O'), ('(', 'O'), ('Falco', 'O'), ('cuvieri', 'O'), (')', 'O'), ('oder', 'O'), ('der', 'O'), ('Malaienbaumfalke', 'O'), ('(', 'O'), ('Falco', 'O'), ('serverus', 'O'), (')', 'O'), ('dieser', 'O'), ('Gruppe', 'O'), ('zuzuzahlen', 'O'), ('sind', 'O'), (',', 'O'), ('ist', 'O'), ('Gegenstand', 'O')]"
1,"[('NEWSru.ua', 'B-OTH'), ('/', 'O'), (':', 'O'), ('оииоо', 'B-OTH'), ('оаа', 'I-OTH'), ('оро', 'I-OTH'), ('в', 'I-OTH'), ('оаве', 'I-OTH'), ('Die', 'I-OTH'), ('SPU', 'O'), ('legte', 'B-ORG'), (',', 'O'), ('wie', 'O'), ('auch', 'O'), ('vier', 'O'), ('weitere', 'O'), ('Parteien', 'O'), (',', 'O'), ('beim', 'O'), ('Obersten', 'O'), ('Gericht', 'O'), ('der', 'O'), ('Ukraine', 'O'), ('Beschwerde', 'B-LOC'), ('gegen', 'O'), ('den', 'O'), ('Ablauf', 'O'), ('der', 'O'), ('Wahl', 'O'), ('ein', 'O'), ('und', 'O'), ('behauptet', 'O'), (',', 'O'), ('es', 'O'), ('sei', 'O'), ('bei', 'O'), ('der', 'O'), ('Auszahlung', 'O'), ('zu', 'O'), ('Unregelmaßigkeit', 'O')]"


In [ ]:
#slow
#hide_input
test_results_df = pd.DataFrame(test_results, columns=['arch', 'tokenizer', 'model_name', 'result', 'error'])
display_df(test_results_df)

,arch,tokenizer,model_name,result,error
0,albert,AlbertTokenizer,albert-base-v1,PASSED,
1,bert,BertTokenizer,bert-base-multilingual-cased,PASSED,
2,camembert,CamembertTokenizer,camembert-base,PASSED,
3,distilbert,DistilBertTokenizer,distilbert-base-uncased,PASSED,
4,electra,ElectraTokenizer,monologg/electra-small-finetuned-imdb,PASSED,
5,longformer,LongformerTokenizer,allenai/longformer-base-4096,PASSED,
6,mobilebert,MobileBertTokenizer,google/mobilebert-uncased,PASSED,
7,roberta,RobertaTokenizer,roberta-base,PASSED,
8,xlm,XLMTokenizer,xlm-mlm-en-2048,PASSED,
9,xlm_roberta,XLMRobertaTokenizer,xlm-roberta-base,PASSED,


## Cleanup

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_data-core.ipynb.
Converted 01a_data-token-classification.ipynb.
Converted 01b_data-question-answering.ipynb.
Converted 01e_data-summarization.ipynb.
Converted 01z_data-language-modeling.ipynb.
Converted 02_modeling-core.ipynb.
Converted 02a_modeling-token-classification.ipynb.
Converted 02b_modeling-question-answering.ipynb.
Converted 02e_modeling-summarization.ipynb.
Converted 02z_modeling-language-modeling.ipynb.
Converted index.ipynb.
